In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./datasets/DfCleaned.csv')
df.head()

,Unnamed: 0,resp_text,clarity,Class,tokens,lemma,ngram2,ngram3
0,0,Prezado Sr Jose Taunai Em atenção ao seu pe...,c5,2,"['prezado', 'sr', 'jose', 'taunai', 'em', 'ate...",prezar sr jose taunai em atenção a o seu pedir...,"[('prezar', 'sr'), ('sr', 'jose'), ('jose', 't...","[('prezar', 'sr', 'jose'), ('sr', 'jose', 'tau..."
1,1,"""A pedido do Pró-Reitor de Graduação, informa...",c5,2,"['a', 'pedido', 'do', 'próreitor', 'de', 'grad...",o pedir de o próreitor de graduação informamo ...,"[('o', 'pedir'), ('pedir', 'de'), ('de', 'o'),...","[('o', 'pedir', 'de'), ('pedir', 'de', 'o'), (..."
2,2,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,1,"['prezado', 'a', 'sr', 'a', 'agradecemos', 'o'...",prezar o sr o agradecer o contato e informamo ...,"[('prezar', 'o'), ('o', 'sr'), ('sr', 'o'), ('...","[('prezar', 'o', 'sr'), ('o', 'sr', 'o'), ('sr..."
3,3,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,1,"['prezado', 'a', 'sr', 'a', 'agradecemos', 'o'...",prezar o sr o agradecer o contato e informamo ...,"[('prezar', 'o'), ('o', 'sr'), ('sr', 'o'), ('...","[('prezar', 'o', 'sr'), ('o', 'sr', 'o'), ('sr..."
4,4,"""Prezado Prof. Gilberto Tadeu Reis da Silva ...",c234,1,"['prezado', 'prof', 'gilberto', 'tadeu', 'reis...",prezar prof gilberto tader rei de o Silva em a...,"[('prezar', 'prof'), ('prof', 'gilberto'), ('g...","[('prezar', 'prof', 'gilberto'), ('prof', 'gil..."


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

2023-11-09 19:21:54.333636: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-09 19:21:54.602427: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-09 19:21:54.602457: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-09 19:21:54.603600: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-09 19:21:54.693818: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-09 19:21:54.694737: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [6]:
max_features = 1000
tokenizer = Tokenizer(num_words= max_features, split= ' ')
tokenizer.fit_on_texts(df['ngram3'].values)
x = tokenizer.texts_to_sequences(df['ngram3'].values)
x = pad_sequences(x)

In [ ]:
embed_dim = 32
lstm_out = 32

model = Sequential([
    layers.Embedding(max_features, embed_dim, input_length= x.shape[1]),
    layers.Conv1D(filters=32, kernel_size=3, padding='same', activation= 'relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.LSTM(128, dropout=0.4, recurrent_dropout=0.4),
    layers.Dense(3, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [8]:
y = pd.get_dummies(df['clarity']).values

### Model

In [10]:
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import LSTM, Dense
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier

In [11]:
def modelLSTM():
    max_features = 1000
    embed_dim = 32
    lstm_out = 32
    
    model = Sequential([
        layers.Embedding(max_features, embed_dim, input_length= x.shape[1]),
        layers.Conv1D(filters=32, kernel_size=3, padding='same', activation= 'relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.LSTM(128, dropout=0.4, recurrent_dropout=0.1),
        layers.Dense(3, activation='softmax')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [14]:
clf = KerasClassifier(build_fn=modelLSTM, epochs=10, batch_size=32, verbose=1)


In [15]:
result = cross_val_score(clf, x, y, scoring='accuracy', cv=10)
result

/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/10


2023-11-09 19:26:57.719528: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 108324000 exceeds 10% of free system memory.


169/169 [==============================] - 435s 3s/step - loss: 0.6268 - accuracy: 0.3876
Epoch 2/10
169/169 [==============================] - 435s 3s/step - loss: 0.5817 - accuracy: 0.4778
Epoch 3/10
169/169 [==============================] - 425s 3s/step - loss: 0.5544 - accuracy: 0.5359
Epoch 4/10
 96/169 [================>.............] - ETA: 3:09 - loss: 0.5265 - accuracy: 0.5762

In [ ]:
result.mean()